# Processing genomic data in Spark

*You can import this notebook from [here](https://github.com/evodify/genomic-analyses_in_apache-spark/tree/master/vcf_filtering_tutorial).*

## [Big Data: Astronomical or Genomical?](http://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.1002195)

### Abstract

Genomics is a Big Data science and is going to get much bigger, very soon, but it is not known whether the needs of genomics will exceed other Big Data domains. Projecting to the year 2025, we compared genomics with three other major generators of Big Data: astronomy, YouTube, and Twitter. Our estimates show that genomics is a “four-headed beast”—it is either on par with or the most demanding of the domains analyzed here in terms of data acquisition, storage, distribution, and analysis. We discuss aspects of new technologies that will need to be developed to rise up and meet the computational challenges that genomics poses for the near future. Now is the time for concerted, community-wide planning for the “genomical” challenges of the next decade.

<img src="http://journals.plos.org/plosbiology/article/figure/image?size=large&id=10.1371/journal.pbio.1002195.g001" width="800">

The plot shows the growth of DNA sequencing both in the total number of human genomes sequenced (left axis) as well as the worldwide annual sequencing capacity (right axis: Tera-basepairs (Tbp), Peta-basepairs (Pbp), Exa-basepairs (Ebp), Zetta-basepairs (Zbps)).


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/Historic_cost_of_sequencing_a_human_genome.svg/800px-Historic_cost_of_sequencing_a_human_genome.svg.png" width="800">

## DNA sequencing

<img src="https://www.yourgenome.org/sites/default/files/illustrations/process/physical_mapping_STS_yourgenome.png" width="800">

## Alligning millions of small DNA sequences (reads) to a reference genome 

<img src="https://github.com/evodify/genomic-analyses_in_apache-spark/raw/master/vcf_filtering_tutorial/genome_read_mapping.png" width="1000">

# VCF

<img src="https://hail.is/docs/stable/_images/hail-vds-rep.png" width="600">

In [5]:
%scala

// This allows easy embedding of publicly available information into any other notebook
// when viewing in git-book just ignore this block - you may have to manually chase the URL in frameIt("URL").
// Example usage:
// displayHTML(frameIt("https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation#Topics_in_LDA",250))
def frameIt( u:String, h:Int ) : String = {
      """<iframe 
 src=""""+ u+""""
 width="95%" height="""" + h + """"
 sandbox>
  <p>
    <a href="http://spark.apache.org/docs/latest/index.html">
      Fallback link for browsers that, unlikely, don't support frames
    </a>
  </p>
</iframe>"""
   }
displayHTML(frameIt("https://en.wikipedia.org/wiki/Variant_Call_Format",500))


Fallback link for browsers that, unlikely, don't support frames

# Hail

[Hail](https://hail.is) is an open-source, scalable framework for exploring and analyzing genomic data. Its functionality is exposed through **Python** and backed by distributed algorithms built on top of **Apache Spark** to efficiently analyze gigabyte-scale data on a laptop or terabyte-scale data on a cluster, without the need to manually chop up data or manage job failures. Users can script pipelines or explore data interactively through **Jupyter notebooks** that flow between Hail with methods for genomics, *PySpark* with scalable *SQL* and *machine learning algorithms*, and *pandas* with *scikit-learn* and *Matplotlib* for results that fit on one machine. Hail also provides a flexible domain language to express complex quality control and analysis pipelines with concise, readable code.

#### Scaling Genetic Data Analysis with Apache Spark
[![Scaling Genetic Data Analysis with Apache Spark](http://img.youtube.com/vi/pyeQusIN5Ao/0.jpg)](https://www.youtube.com/embed/pyeQusIN5Ao)

## This Notebook is based on the tutorial [Analyzing 1000 Genomes with Spark and Hail](https://docs.databricks.com/spark/latest/training/1000-genomes.html)

## Cluster setup

First download Hail's Python and Java libraries to your computer:

https://storage.googleapis.com/hail-common/hail-tutorial-databricks.jar

https://storage.googleapis.com/hail-common/hail-devel-py2.7-databricks.egg

Then on the Databricks interface, navigate to `Workspace > Users > Username` and select `Import` from the Username drop-down menu. At the bottom of `Import Notebooks` window, click the link in `(To import a library, such as a jar or egg,`_`click here`_`)`.  Upload both the .jar and .egg files using this interface, using any names you like. Make sure that the option `Attach automatically to all clusters` is checked in the success dialog.

Next click the `Clusters` icon on the left sidebar and then `+Create Cluster`. For `Apache Spark Version`, select `Spark 2.0 (Auto-updating, Scala 2.11)`. Note that Hail won't work with Scala 2.10! In the Databricks cluster creation dialog, click `Show advanced settings` at bottom and then on the `Spark` tab, and paste the text below into the `Spark config` box.

```
spark.hadoop.io.compression.codecs org.apache.hadoop.io.compress.DefaultCodec,is.hail.io.compress.BGzipCodec,org.apache.hadoop.io.compress.GzipCodec
spark.sql.files.openCostInBytes 1099511627776
spark.sql.files.maxPartitionBytes 1099511627776
spark.hadoop.mapreduce.input.fileinputformat.split.minsize 1099511627776
spark.hadoop.parquet.block.size 1099511627776```
Start the cluster and attach this notebook to it by clicking on your cluster name in menu `Detached` at the top left of this workbook. Now you're ready to Hail!

In [9]:
from hail import *
hc = HailContext(sc)

In [10]:
#

In [11]:

# these are the methods possible for hc ...
%python
for i in dir(hc):  
  if not i.startswith("_"):
    print(i)
    

ERROR: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).
balding_nichols_model
dataframe_to_keytable
grep
import_annotations_table
import_bgen
import_gen
import_keytable
import_plink
import_vcf
index_bgen
read
sc
stop
write_partitioning

Let's import some Python libraries for use throughout the tutorial.

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from math import log, isnan
import seaborn

## Import data

We must first import variant data into Hail's internal format of Variant Dataset (VDS). We use the [import_vcf](https://hail.is/hail/hail.HailContext.html#hail.HailContext.import_vcf) method on [HailContext](https://hail.is/hail/hail.HailContext.html) to load a VCF file into Hail.

It is recommended to load a block-compressed VCF (`.vcf.bgz`) which enables Hail to read the file in parallel. Reading files that have not been block-compressed (`.vcf`, `.vcf.gz`) is _significantly_ slower and should be avoided (though often `.vcf.gz` files are in fact block-compressed, in which case renaming to `.vcf.bgz` solves the problem).

Unfortunately, I was not able to load `.bgz` compressed file. It worked for the existing human dataset, which is bgz compressed, but not for my data (See below).

### Cbp data
As a training data set, I use a subset of 1% from my unpublished genomic data on [*Capsella bursa-pastoris*](https://en.wikipedia.org/wiki/Capsella_bursa-pastoris) (hereafter Cbp).

### Human data

You can use publicly avaliable human data by uncommenting the three line below. Jump to Cmd 19.

## Download the data

In [16]:
%sh
wget https://github.com/evodify/genomic-analyses_in_apache-spark/raw/master/vcf_filtering_tutorial/Cbp31_SNPs_test0.01.vcf.gz # download the test VCF file
wget https://github.com/evodify/genomic-analyses_in_apache-spark/raw/master/vcf_filtering_tutorial/Cbp31_annot.csv # download the test annotation file
gunzip Cbp31_SNPs_test0.01.vcf.gz # uncomress the gzip file.


--2019-06-24 20:47:08-- https://github.com/evodify/genomic-analyses_in_apache-spark/raw/master/vcf_filtering_tutorial/Cbp31_SNPs_test0.01.vcf.gz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/evodify/genomic-analyses_in_apache-spark/master/vcf_filtering_tutorial/Cbp31_SNPs_test0.01.vcf.gz [following]
--2019-06-24 20:47:08-- https://raw.githubusercontent.com/evodify/genomic-analyses_in_apache-spark/master/vcf_filtering_tutorial/Cbp31_SNPs_test0.01.vcf.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.40.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.40.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27186220 (26M) [application/octet-stream]
Saving to: ‘Cbp31_SNPs_test0.01.vcf.gz’

 0K .......... .......... .......... .......... .......... 0% 1.10M 24s
 50K .......... .......... .......... .......... .......... 0% 3.10M 16s
 100K .......... .......... .......... .......... .......... 0% 7.19M 12s
 150K .......... .......... .......... .......... .......... 0% 3.63M 11s
 200K .......... .......... .......... .......... .......... 0% 5.73M 9s
 250K .......... .......... .......... .......... .......... 1% 4.58M 9s
 300K .......... .......... .......... .......... .......... 1% 4.22M 8s
 350K .......... .......... .......... .......... .......... 1% 4.83M 8s
 400K .......... .......... .......... .......... .......... 1% 6.37M 7s
 450K .......... .......... .......... .......... .......... 1% 4.98M 7s
 500K .......... .......... .......... .......... .......... 2% 6.07M 7s
 550K .......... .......... .......... .......... .......... 2% 6.72M 7s
 600K .......... .......... .......... .......... .......... 2% 5.43M 6s
 650K .......... .......... .......... .......... .......... 2% 5.84M 6s
 700K .......... .......... .......... .......... .......... 2% 4.43M 6s
 750K .......... .......... .......... .......... .......... 3% 6.61M 6s
 800K .......... .......... .......... .......... .......... 3% 6.10M 6s
 850K .......... .......... .......... .......... .......... 3% 7.22M 6s
 900K .......... .......... .......... .......... .......... 3% 6.85M 6s
 950K .......... .......... .......... .......... .......... 3% 5.71M 6s
 1000K .......... .......... .......... .......... .......... 3% 7.60M 6s
 1050K .......... .......... .......... .......... .......... 4% 8.33M 5s
 1100K .......... .......... .......... .......... .......... 4% 4.85M 5s
 1150K .......... .......... .......... .......... .......... 4% 8.37M 5s
 1200K .......... .......... .......... .......... .......... 4% 6.51M 5s
 1250K .......... .......... .......... .......... .......... 4% 7.73M 5s
 1300K .......... .......... .......... .......... .......... 5% 9.41M 5s
 1350K .......... .......... .......... .......... .......... 5% 5.93M 5s
 1400K .......... .......... .......... .......... .......... 5% 8.09M 5s
 1450K .......... .......... .......... .......... .......... 5% 9.43M 5s
 1500K .......... .......... .......... .......... .......... 5% 5.85M 5s
 1550K .......... .......... .......... .......... .......... 6% 7.91M 5s
 1600K .......... .......... .......... .......... .......... 6% 10.5M 5s
 1650K .......... .......... .......... .......... .......... 6% 8.72M 5s
 1700K .......... .......... .......... .......... .......... 6% 7.43M 5s
 1750K .......... .......... .......... .......... .......... 6% 8.59M 4s
 1800K .......... .......... .......... .......... .......... 6% 10.8M 4s
 1850K .......... .......... .......... .......... .......... 7% 7.99M 4s
 1900K .......... .......... .......... .......... .......... 7% 6.98M 4s
 1950K .......... .......... .......... .......... .......... 7% 9.24M 4s
 2000K .......... .......... .......... .......... .......... 7% 11.5M 4s
 2050K .......... .......... .....

In [17]:
%sh pwd && ls -l

/databricks/driver
total 104316
-rw-r--r-- 1 root root 712 Jun 24 20:26 Cbp31_annot.csv
-rw-r--r-- 1 root root 712 Jun 24 20:38 Cbp31_annot.csv.1
-rw-r--r-- 1 root root 712 Jun 24 20:47 Cbp31_annot.csv.2
-rw-r--r-- 1 root root 106677117 Jun 24 20:47 Cbp31_SNPs_test0.01.vcf
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
-rw-r--r-- 1 root root 731 Jun 24 20:24 derby.log
drwxr-xr-x 3 root root 4096 Jun 24 20:24 eventlogs
drwxr-xr-x 2 root root 4096 Jun 24 20:45 ganglia
-rw-r--r-- 1 root root 103281 Jun 24 20:47 hail.log
drwxr-xr-x 2 root root 4096 Jun 24 20:24 logs

## Move the downloaded data to DBFS

In [19]:

dbutils.fs.mkdirs("dbfs:/FileStore/tables/Cbp") # create a new directory for our data files


Out[ 59 ]: True

In [20]:
dir(dbutils)

Out[ 60 ]: 
['CredentialsHandler',
 'FSHandler',
 'LibraryHandler',
 'NotebookHandler',
 'PreviewHandler',
 'SecretsHandler',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'entry_point',
 'fs',
 'help',
 'shell',
 'widgets']

In [21]:

dbutils.fs.mv("file:/databricks/driver/Cbp31_SNPs_test0.01.vcf", "dbfs:/FileStore/tables/Cbp") # move
dbutils.fs.cp("file:/databricks/driver/Cbp31_annot.csv", "dbfs:/FileStore/tables/Cbp") # copy because we will need non-dbfs file for R later


Out[ 61 ]: True

In [22]:

dbutils.fs.ls("dbfs:/FileStore/tables/Cbp/")


Out[ 62 ]: 
[FileInfo(path='dbfs:/FileStore/tables/Cbp/Cbp31_SNPs_test0.01.vcf', name='Cbp31_SNPs_test0.01.vcf', size=106677117),
 FileInfo(path='dbfs:/FileStore/tables/Cbp/Cbp31_annot.csv', name='Cbp31_annot.csv', size=712)]

## Start processing

In [24]:
vcf_path = '/FileStore/tables/Cbp/Cbp31_SNPs_test0.01.vcf'
annotation_path = '/FileStore/tables/Cbp/Cbp31_annot.csv'

# comment out the line above and uncommend the lines below for human data

# vcf_path = '/databricks-datasets/hail/data-001/1kg_sample.vcf.bgz'
# annotation_path = '/databricks-datasets/hail/data-001/1kg_annotations.txt'
# purcell_5k_path = '/databricks-datasets/hail/data-001/purcell5k.interval_list'

In [25]:
vds = hc.import_vcf(vcf_path) # bgz import fails even with force_bgz=True. I compressed my files with bgzip from https://github.com/samtools/tabix .

This method produced a [VariantDataset](https://hail.is/hail/hail.VariantDataset.html), Hail's primary representation of genomic data. Following that link to Hail's python API documentation will let you see the myriad methods it offers.  We will use but a few of them in this tutorial. We next use its [split_multi](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.split_multi) to split multi-allelic variants into biallelic variants. For example, the variant `1:1000:A:T,C` would become two variants: `1:1000:A:T` and `1:1000:A:C`.

In [27]:
vds = vds.split_multi()

In [28]:
print(type(vds))

<class 'hail.dataset.VariantDataset'>

In [29]:

dir(vds)


Out[ 67 ]: 
['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_global_schema',
 '_globals',
 '_jvds',
 '_num_samples',
 '_sa_schema',
 '_sample_annotations',
 '_sample_ids',
 '_typecheck',
 '_va_schema',
 'aggregate_by_key',
 'aggregate_intervals',
 'annotate_alleles_expr',
 'annotate_global_list',
 'annotate_global_py',
 'annotate_global_table',
 'annotate_samples_expr',
 'annotate_samples_fam',
 'annotate_samples_list',
 'annotate_samples_table',
 'annotate_samples_vds',
 'annotate_variants_bed',
 'annotate_variants_expr',
 'annotate_variants_intervals',
 'annotate_variants_keytable',
 'annotate_variants_loci',
 'annotate_variants_table',
 'annotate_variants_vds',
 'cache',
 'concordance',
 'count',
 'count_variants',
 'deduplicate',
 'downsample_variants',
 'export_gen',
 'export_genotypes',
 'export_plink',
 'export_samples',
 'export_variants',
 'export_variants_cass',
 'export_variants_solr',
 'export_vcf',
 'file_version',
 'filter_alleles',
 'filter_genotypes',
 'filter_multi',
 'filter_samples_all',
 'filter_samples_expr',
 'filter_samples_list',
 'filter_variants_all',
 'filter_variants_expr',
 'filter_variants_intervals',
 'filter_variants_list',
 'global_schema',
 'globals',
 'grm',
 'hardcalls',
 'hc',
 'ibd',
 'impute_sex',
 'is_dosage',
 'join',
 'linreg',
 'lmmreg',
 'logreg',
 'make_keytable',
 'mendel_errors',
 'min_rep',
 'num_partitions',
 'num_samples',
 'pca',
 'persist',
 'query_samples',
 'query_samples_typed',
 'query_variants',
 'query_variants_typed',
 'rename_samples',
 'repartition',
 'same',
 'sample_annotations',
 'sample_ids',
 'sample_qc',
 'sample_schema',
 'samples_keytable',
 'sparkinfo',
 'split_multi',
 'tdt',
 'variant_qc',
 'variant_schema',
 'variants_keytable',
 'vep',
 'was_split',
 'write']

We next use the [annotate_samples_table](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.annotate_samples_table) method to load phenotypic information on each sample from the sample annotations file.

Here `annotation_path` refers to the sample annotation data file, whose first few lines are:
```
Sample Population
DL174     ASI
GY37      ASI
HJC419    ASI
12.4      EUR
13.16     EUR
16.9      EUR
BEL5      EUR
```

The `root` argument says where to put this data. For sample annotations, the root must start with `sa` followed by a `.` and the rest is up to you, so let's use `sa.myAnnot`.

The `sample_expr` argument indicates that the sample ID is in column `Sample`.

The object `TextTableConfig` allows users to provide information about column data types, header existence, comment characters, and field delimiters. For example,  'Population: Boolean'.  `impute=True` will infer column types automatically.

In [31]:
dbutils.fs.head(annotation_path)

Out[ 68 ]: 'Sample\tPopulation\tLatitude\tLongitude\nDL174\tASI\t38.56\t121.35\nGY37\tASI\t26.37\t106.43\nHJC419\tASI\t30.16\t120.13\nHY85\tASI\t26.53\t112.33\nJZH152\tASI\t30.20\t112.06\nKMB206\tASI\t25.06\t102.41\nNJ219\tASI\t32.03\t118.46\nPL\tASI\t23.97\t120.95\nTBS195\tASI\t33.57\t107.45\nTY118\tASI\t37.55\t112.32\nXN444\tASI\t36.37\t101.46\n12.4\tEUR\t51.50\t11.97\n13.16\tEUR\t50.06\t19.94\n16.9\tEUR\t51.81\t5.84\nBEL5\tEUR\t50.55\t128.28\nFR50\tEUR\t48.08\t7.37\nHRB132\tEUR\t45.45\t126.37\nIRRU2\tEUR\t52.16\t104.18\nRK32\tEUR\t64.13\t-21.82\nSE14\tEUR\t62.64\t17.94\nSE33\tEUR\t56.15\t13.77\nSTA4\tEUR\t56.20\t-2.47\nSTJ2\tEUR\t44.51\t-1.21\nVLA3\tEUR\t43.13\t131.40\n39-12-28\tME\t44.40\t7.40\n5.16\tME\t41.65\t-4.72\n70.5\tME\t37.97\t24.00\nAL87\tME\t35.45\t7.96\nJO56\tME\t31.97\t35.98\nTR73\tME\t41.02\t28.97\nWAC5\tME\t31.29\t-97.17\n'

In [32]:

vds = vds.annotate_samples_table(annotation_path,
                                 root='sa.myAnnot',
                                 sample_expr='Sample',
                                 config=TextTableConfig(impute=True))


Lastly, we'll [write](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.write) the dataset to disk so that all future computations begin by reading in the fast VDS rather than the slow VCF.

In [34]:
out_path = '/cbp.vds'
vds.write(out_path, overwrite=True)


## Start exploring

Now we're ready to start exploring! We will read back in the VDS we wrote to disk:

In [36]:
vds

Out[ 71 ]: <hail.dataset.VariantDataset at 0x7f468c109c18>

In [37]:
hc.sc

Out[ 72 ]: <SparkContext master=local[8] appName=Databricks Shell>

In [38]:
print(out_path)

/cbp.vds

In [39]:
import py4j

In [40]:
vds = hc.read(out_path)

--------------------------------------------------------------------------- 
 FatalError Traceback (most recent call last)
 <command-3160555503398378> in <module> () 
 ----> 1 vds = hc . read ( out_path ) 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/context.py in read (self, path, sites_only) 
 454 if sites_only : 
 455 pargs . append ( "--skip-genotypes" ) 
 --> 456 return self . _run_command ( None , pargs ) 
 457 
 458 def write_partitioning ( self , path ) : 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/context.py in _run_command (self, vds, pargs) 
 77 result = cmd . run ( jstate , cmd_args ) 
 78 except Py4JJavaError as e : 
 ---> 79 raise_py4j_exception ( e ) 
 80 
 81 return VariantDataset ( self , result . vds ( ) ) 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/java.py in raise_py4j_exception (e) 
 85 def raise_py4j_exception ( e ) : 
 86 msg = env . jutils . getMinimalMessage ( e . java_exception ) 
 ---> 87 raise FatalError ( msg , e . java_exception ) 

 FatalError : 
corrupt VDS: invalid metadata file.
 Recreate VDS with current version of Hail.
 caught exception: java.lang.NoSuchMethodError: org.json4s.jackson.JsonMethods$.parse(Lorg/json4s/JsonInput;Z)Lorg/json4s/JsonAST$JValue;
	at is.hail.variant.VariantSampleMatrix$$anonfun$1.apply(VariantSampleMatrix.scala:65)
	at is.hail.variant.VariantSampleMatrix$$anonfun$1.apply(VariantSampleMatrix.scala:65)
	at is.hail.utils.richUtils.RichHadoopConfiguration$.readFile$extension(RichHadoopConfiguration.scala:243)
	at is.hail.variant.VariantSampleMatrix$.readMetadata(VariantSampleMatrix.scala:65)
	at is.hail.variant.VariantSampleMatrix$.read(VariantSampleMatrix.scala:171)
	at is.hail.driver.Read$$anonfun$1.apply(Read.scala:41)
	at is.hail.driver.Read$$anonfun$1.apply(Read.scala:41)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:186)
	at is.hail.driver.Read$.run(Read.scala:41)
	at is.hail.driver.Read$.run(Read.scala:9)
	at is.hail.driver.Command.runCommand(Command.scala:256)
	at is.hail.driver.Command.run(Command.scala:261)
	at sun.reflect.GeneratedMethodAccessor434.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

First, we'll print some statistics about the size of the dataset using [count](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.count):

In [43]:
print(vds.count())

{'nSamples': 31, 'nVariants': 98094, 'nGenotypes': 3040914}

If the Boolean parameter `genotypes` is set to `True`, the overall call rate across all genotypes is computed as well:

In [45]:
vds.count(genotypes=True)

Out[ 81 ]: {'nGenotypes': 3040914, 'nVariants': 98094, 'nSamples': 31, 'nCalled': 2574591, 'callRate': 84.66503820890692}

So the call rate before any QC filtering is about 84.67%.

Let's print the types of all annotations.

Variant annotations:

In [47]:
print(vds.variant_schema)

Struct {
 rsid: String,
 qual: Double,
 filters: Set[String],
 pass: Boolean,
 info: Struct {
 AC: Array[Int],
 AF: Array[Double],
 AN: Int,
 BaseQRankSum: Double,
 ClippingRankSum: Double,
 DP: Int,
 DS: Boolean,
 END: Int,
 ExcessHet: Double,
 FS: Double,
 HaplotypeScore: Double,
 InbreedingCoeff: Double,
 MLEAC: Array[Int],
 MLEAF: Array[Double],
 MQ: Double,
 MQRankSum: Double,
 QD: Double,
 RAW_MQ: Double,
 ReadPosRankSum: Double,
 SOR: Double
 },
 aIndex: Int,
 wasSplit: Boolean
}

Sample annotations:

In [49]:
print(vds.sample_schema)

Struct {
 myAnnot: Struct {
 Sample: String,
 Population: String,
 Latitude: Double,
 Longitude: Double
 }
}

We have just printed the sample and variant structure above. Global structure is empty.
 
 To recall the structure:

<img src="https://hail.is/docs/stable/_images/hail-vds-rep.png" width="600">

Also, note the annotations imported from the original VCF, as well as the sample annotations added above. Notice how those six sample annotations loaded above are nested inside `sa.structure` as defined by the `root` option in [annotate_samples_table](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.annotate_samples_table).

Next we'll add some global annotations including the list of populations that are present in our dataset and the number of samples in each population, using the Hail expression language and the [query_samples](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.query_samples) method. The codings are:

  - ASI = Asian
  - EUR = European
  - ME = Middle Eastern

We'll first build up a list of query expressions, then evaluate them all at once to save time.

In [52]:
expressions = ['samples.map(s => sa.myAnnot.Population).collect().toSet']
queries = vds.query_samples(expressions)    # popping ! ! ! !  something wrong...

print('populations = %s' % queries[0])
print('total samples = %s' % vds.num_samples)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3160555503398389> in <module> () 
 1 expressions = [ 'samples.map(s => sa.myAnnot.Population).collect().toSet' ] 
 ----> 2 queries = vds . query_samples ( expressions ) # popping ! ! ! ! something wrong... 
 3 
 4 print ( 'populations = %s' % queries [ 0 ] ) 
 5 print ( 'total samples = %s' % vds . num_samples ) 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/dataset.py in query_samples (self, exprs) 
 2797 : rtype : list 
 2798 """
 -> 2799 r , t = self . query_samples_typed ( exprs ) 
 2800 return r 
 2801 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/dataset.py in query_samples_typed (self, exprs) 
 2761 result_list = self . _jvds . querySamples ( jarray ( env . jvm . java . lang . String , exprs ) ) 
 2762 ptypes = [ Type . _from_java ( x . _2 ( ) ) for x in result_list ] 
 -> 2763 annotations = [ ptypes [ i ] . _convert_to_py ( result_list [ i ] . _1 ( ) ) for i in xrange ( len ( ptypes ) ) ] 
 2764 return annotations , ptypes 
 2765 

 NameError : name 'xrange' is not defined

Now it's easy to count samples by population using the [counter](https://hail.is/expr_lang.html#counter) aggregator:

In [54]:
counter = vds.query_samples('samples.map(s => sa.myAnnot.Population).counter()')[0]
for x in counter:
    print('population %s found %s times' % (x.key, x.count))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3160555503398391> in <module> () 
 ----> 1 counter = vds . query_samples ( 'samples.map(s => sa.myAnnot.Population).counter()' ) [ 0 ] 
 2 for x in counter : 
 3 print ( 'population %s found %s times' % ( x . key , x . count ) ) 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/dataset.py in query_samples (self, exprs) 
 2797 : rtype : list 
 2798 """
 -> 2799 r , t = self . query_samples_typed ( exprs ) 
 2800 return r 
 2801 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/dataset.py in query_samples_typed (self, exprs) 
 2761 result_list = self . _jvds . querySamples ( jarray ( env . jvm . java . lang . String , exprs ) ) 
 2762 ptypes = [ Type . _from_java ( x . _2 ( ) ) for x in result_list ] 
 -> 2763 annotations = [ ptypes [ i ] . _convert_to_py ( result_list [ i ] . _1 ( ) ) for i in xrange ( len ( ptypes ) ) ] 
 2764 return annotations , ptypes 
 2765 

 NameError : name 'xrange' is not defined

## Quality control (QC)

VCF file contains many annotations scores that define the quality of genotypes as well as quality of a variant.

### Filter genotypes

Let's filter genotypes based on genotype quality (GQ) and read coverage (DP).

Here `g` is genotype, `v` is variant, `s` is sample, and annotations are accessible via `va`, `sa`, and `global`.

In [56]:
%scala

// This allows easy embedding of publicly available information into any other notebook
// when viewing in git-book just ignore this block - you may have to manually chase the URL in frameIt("URL").
// Example usage:
// displayHTML(frameIt("https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation#Topics_in_LDA",250))
def frameIt( u:String, h:Int ) : String = {
      """<iframe 
 src=""""+ u+""""
 width="95%" height="""" + h + """"
 sandbox>
  <p>
    <a href="http://spark.apache.org/docs/latest/index.html">
      Fallback link for browsers that, unlikely, don't support frames
    </a>
  </p>
</iframe>"""
   }
displayHTML(frameIt("https://en.wikipedia.org/wiki/Phred_quality_score",500))

In [57]:
filter_condition_gDP_gGQ = 'g.dp >= 10 && g.gq >= 20'
vds_gDP_gGQ = vds.filter_genotypes(filter_condition_gDP_gGQ)

In [58]:
vds_gDP_gGQ.count(genotypes=True)

Out[ 87 ]: {'nGenotypes': 3040914, 'nVariants': 98094, 'nSamples': 31, 'nCalled': 1532341, 'callRate': 50.39080355445764}

Now the call rate is about 50%, so nearly 35% of genotypes failed the filter. Filtering out a genotype is equivalent to setting the genotype call to missing.

Having removed suspect genotypes, let's next remove variants with low call rate and then calculate summary statistics per sample with the [sample_qc](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.sample_qc) method.

In [60]:
vds_gDP_gGQ_vCR = (vds_gDP_gGQ
    .filter_variants_expr('gs.fraction(g => g.isCalled) >= 0.50')
    .sample_qc())

Check how many variants retained after filtering.

In [62]:
vds_gDP_gGQ_vCR.count(genotypes=True)

Out[ 89 ]: {'nGenotypes': 1549783, 'nVariants': 49993, 'nSamples': 31, 'nCalled': 1300202, 'callRate': 83.8957454043566}

### Filter samples

The call rate for each variant is calculated using the `fraction` [aggregable](https://hail.is/expr_lang.html#aggregables) on the genotypes `gs`. [sample_qc](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.sample_qc) adds a number of statistics to sample annotations:

In [65]:
print(vds_gDP_gGQ_vCR.sample_schema)

Struct {
 myAnnot: Struct {
 Sample: String,
 Population: String,
 Latitude: Double,
 Longitude: Double
 },
 qc: Struct {
 callRate: Double,
 nCalled: Int,
 nNotCalled: Int,
 nHomRef: Int,
 nHet: Int,
 nHomVar: Int,
 nSNP: Int,
 nInsertion: Int,
 nDeletion: Int,
 nSingleton: Int,
 nTransition: Int,
 nTransversion: Int,
 dpMean: Double,
 dpStDev: Double,
 gqMean: Double,
 gqStDev: Double,
 nNonRef: Int,
 rTiTv: Double,
 rHetHomVar: Double,
 rInsertionDeletion: Double
 }
}

Let's export these sample annotations to a text file and take a look at them:

In [67]:
vds_gDP_gGQ_vCR.export_samples('file:///sampleqc.txt', 'Sample = s.id, sa.qc.*')

In [68]:
%sh
head /sampleqc.txt | cut -f 1-8 | column -t

Sample callRate nCalled nNotCalled nHomRef nHet nHomVar nSNP
12.4 9.08767e-01 45432 4561 16033 23357 6042 35441
13.16 9.13688e-01 45678 4315 16738 23586 5354 34294
16.9 9.21269e-01 46057 3936 16748 23511 5798 35107
39-12-28 9.17848e-01 45886 4107 15673 23725 6488 36701
5.16 9.04287e-01 45208 4785 15547 23301 6360 36021
70.5 9.01606e-01 45074 4919 15853 23251 5970 35191
AL87 7.41064e-01 37048 12945 13679 18281 5088 28457
BEL5 8.56140e-01 42801 7192 15289 22488 5024 32536
DL174 7.95871e-01 39788 10205 13824 19608 6356 32320

We can further analyze these results locally using Python's [matplotlib](http://matplotlib.org/) library. Below is an example plot of three variables (call rate, mean depth and mean GQ), along with the code that generate the plot.

In [70]:
sampleqc_table = vds_gDP_gGQ_vCR.samples_keytable().to_pandas()

plt.clf() # clears the entire current figure with all its axes, but leaves the window opened, such that it may be reused for other plots.
plt.figure(figsize=(7,4)) # figure size in inches; change according to your screen size and resolution

plt.subplot(1, 3, 1)
plt.hist(sampleqc_table["sa.qc.callRate"], bins=np.arange(0.4, 1.01, .05))
plt.xlabel("Call Rate")
plt.ylabel("Frequency")
plt.xlim(0.4, 1)
plt.axvline(.50, color='r')

plt.subplot(1, 3, 2)
plt.hist(sampleqc_table["sa.qc.dpMean"], bins=np.arange(20, 80, 10))
plt.xlabel("Mean depth")
plt.ylabel("Frequency")
plt.xlim(10, 80)
plt.axvline(10, color='r')

plt.subplot(1, 3, 3)
plt.hist(sampleqc_table["sa.qc.gqMean"], bins=np.arange(20, 100, 10))
plt.xlabel("Mean Sample GQ")
plt.ylabel("Frequency")
plt.axvline(30, color = 'r')
plt.xlim(0, 100)

plt.tight_layout()
plt.show()
display()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3160555503398407> in <module> () 
 ----> 1 sampleqc_table = vds_gDP_gGQ_vCR . samples_keytable ( ) . to_pandas ( ) 
 2 
 3 plt . clf ( ) # clears the entire current figure with all its axes, but leaves the window opened, such that it may be reused for other plots. 
 4 plt . figure ( figsize = ( 7 , 4 ) ) # figure size in inches; change according to your screen size and resolution 
 5 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/keytable.py in to_pandas (self, expand, flatten) 
 533 : return : pandas . DataFrame 
 534 """
 --> 535 return self . to_dataframe ( expand , flatten ) . toPandas ( ) 
 536 
 537 def export_mongodb ( self , mode = 'append' ) : 

 /databricks/spark/python/pyspark/sql/dataframe.py in toPandas (self) 
 2127 import pandas as pd 
 2128 
 -> 2129 if self . sql_ctx . _conf . pandasRespectSessionTimeZone ( ) : 
 2130 timezone = self . sql_ctx . _conf . sessionLocalTimeZone ( ) 
 2131 else : 

 /databricks/spark/python/pyspark/sql/context.py in _conf (self) 
 97 def _conf ( self ) : 
 98 """Accessor for the JVM SQL-specific configurations""" 
 ---> 99 return self . sparkSession . _jsparkSession . sessionState ( ) . conf ( ) 
 100 
 101 @ classmethod 

 AttributeError : 'JavaMember' object has no attribute 'sessionState'

You can remove samples that are outliers in the plots above, where cutoffs are given by the red lines. But there are no outliers here. If we had to filter, we could do this step:

In [72]:
vds_gDP_gGQ_vCR_sDP_sGT = (vds_gDP_gGQ_vCR
    .annotate_samples_vds(vds_gDP_gGQ_vCR, code = 'sa.qc = vds.qc' )
    .filter_samples_expr('sa.qc.dpMean > 0.50 && sa.qc.dpMean >=10 && sa.qc.gqMean >= 20 '))

As before, we can count the number of samples that remain in the dataset after filtering. (But nothing has been filtered out here)

In [74]:
vds_gDP_gGQ_vCR_sDP_sGT.count(genotypes=True)

Out[ 95 ]: {'nGenotypes': 1549783, 'nVariants': 49993, 'nSamples': 31, 'nCalled': 1300202, 'callRate': 83.8957454043566}

### Filter variants

We now have `vds_gDP_gGQ_vCR_sDP_sGT`, a VDS where low-quality genotypes and samples have been removed.

Let's use the [variant_qc](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.variant_qc) method to start exploring variant metrics:

In [76]:
vds_gDP_gGQ_vCR_sDP_sGT = vds_gDP_gGQ_vCR_sDP_sGT.variant_qc()
print(vds_gDP_gGQ_vCR_sDP_sGT.variant_schema)

Struct {
 rsid: String,
 qual: Double,
 filters: Set[String],
 pass: Boolean,
 info: Struct {
 AC: Array[Int],
 AF: Array[Double],
 AN: Int,
 BaseQRankSum: Double,
 ClippingRankSum: Double,
 DP: Int,
 DS: Boolean,
 END: Int,
 ExcessHet: Double,
 FS: Double,
 HaplotypeScore: Double,
 InbreedingCoeff: Double,
 MLEAC: Array[Int],
 MLEAF: Array[Double],
 MQ: Double,
 MQRankSum: Double,
 QD: Double,
 RAW_MQ: Double,
 ReadPosRankSum: Double,
 SOR: Double
 },
 aIndex: Int,
 wasSplit: Boolean,
 qc: Struct {
 callRate: Double,
 AC: Int,
 AF: Double,
 nCalled: Int,
 nNotCalled: Int,
 nHomRef: Int,
 nHet: Int,
 nHomVar: Int,
 dpMean: Double,
 dpStDev: Double,
 gqMean: Double,
 gqStDev: Double,
 nNonRef: Int,
 rHeterozygosity: Double,
 rHetHomVar: Double,
 rExpectedHetFrequency: Double,
 pHWE: Double
 }
}

Next, we will filter variants following the [Best Practices GATK recommendations](https://gatkforums.broadinstitute.org/gatk/discussion/2806/howto-apply-hard-filters-to-a-call-set).

These recommendations are for human data, but our data is not human and the distribution of quality statistics will differ from expected for human. (Explain why it is so is beyond the scope of this tutorial.)

Let's have a look at the distribution of different variant quality statistics:

- QD - variant confidence standardized by depth.

  This annotation puts the variant confidence QUAL score into perspective by normalizing for the amount of coverage available. Because each read contributes a little to the QUAL score, variants in region with deep coverage can have artificially inflated QUAL scores, giving the impression that the call is supported by more evidence than it really is. To compensate for this, we normalize the variant confidence by depth, which gives us a more objective picture of how well supported the call is.
   
- MQ - Mapping quality of a SNP.

- FS - strand bias in support for REF vs ALT allele calls.

  Strand bias is a type of sequencing bias in which one DNA strand is favored over the other, which can result in incorrect evaluation of the amount of evidence observed for one allele vs. the other. The FisherStrand annotation is one of several methods that aims to evaluate whether there is strand bias in the data. It uses Fisher's Exact Test to determine if there is strand bias between forward and reverse strands for the reference or alternate allele. The output is a Phred-scaled p-value. The higher the output value, the more likely there is to be bias. More bias is indicative of false positive calls.
  
- SOR - sequencing bias in which one DNA strand is favored over the other
   
   Strand bias is a type of sequencing bias in which one DNA strand is favored over the other, which can result in incorrect evaluation of the amount of evidence observed for one allele vs. the other. It is used to determine if there is strand bias between forward and reverse strands for the reference or alternate allele. The reported value is ln-scaled.
   
- MQRankSum - Rank sum test for mapping qualities of REF vs. ALT reads.
   
   This variant-level annotation compares the mapping qualities of the reads supporting the reference allele with those supporting the alternate allele. The ideal result is a value close to zero, which indicates there is little to no difference. A negative value indicates that the reads supporting the alternate allele have lower mapping quality scores than those supporting the reference allele. Conversely, a positive value indicates that the reads supporting the alternate allele have higher mapping quality scores than those supporting the reference allele.

- ReadPosRankSum - do all the reads support a SNP call tend to be near the end of a read.
   
   The ideal result is a value close to zero, which indicates there is little to no difference in where the alleles are found relative to the ends of reads. A negative value indicates that the alternate allele is found at the ends of reads more often than the reference allele. Conversely, a positive value indicates that the reference allele is found at the ends of reads more often than the alternate allele.

We've once again used matplotlib to make histograms of these siz summary statistics.

In [79]:
variantqc_table = vds_gDP_gGQ_vCR_sDP_sGT.variants_keytable().to_pandas()

plt.clf() # clears the entire current figure with all its axes, but leaves the window opened, such that it may be reused for other plots.
plt.figure(figsize=(7,7)) # figure size in inches; change according to your screen size and resolution

plt.subplot(3, 2, 1)
variantgq_MQ = variantqc_table["va.info.MQ"]
plt.hist(variantgq_MQ.dropna(), bins = np.arange(0, 101, 2)) # It is important to add dropna() to skip NA values. Otherwise, the script won't work.
plt.xlabel("MQ")
plt.ylabel("Frequency")
plt.xlim(0, 100)
plt.axvline(30, color = 'r')

plt.subplot(3, 2, 2)
variantgq_SOR = variantqc_table["va.info.SOR"]
plt.hist(variantgq_SOR.dropna(), bins = np.arange(0, 8, 0.2))
plt.xlabel("SOR")
plt.ylabel("Frequency")
plt.xlim(0, 8)
plt.axvline(4, color = 'r')

plt.subplot(3, 2, 3)
variantgq_QD = variantqc_table["va.info.QD"]
plt.hist(variantgq_QD.dropna(), bins = np.arange(0, 40, 1))
plt.xlabel("QD")
plt.ylabel("Frequency")
plt.xlim(0, 40)
plt.axvline(2, color = 'r')

plt.subplot(3, 2, 4)
variantgq_FS = variantqc_table["va.info.FS"]
plt.hist(variantgq_FS.dropna(), bins = np.arange(0, 100, 2))
plt.xlabel("FS")
plt.ylabel("Frequency")
plt.xlim(0, 100)
plt.axvline(60, color = 'r')

plt.subplot(3, 2, 5)
variantgq_MQRankSum = variantqc_table["va.info.MQRankSum"]
plt.hist(variantgq_MQRankSum.dropna(), bins = np.arange(-20, 20, 1))
plt.xlabel("MQRankSum")
plt.ylabel("Frequency")
plt.xlim(-25, 25)
plt.axvline(-20, color = 'r')

plt.subplot(3, 2, 6)
variantgq_ReadPosRankSum = variantqc_table["va.info.ReadPosRankSum"]
plt.hist(variantgq_ReadPosRankSum.dropna(), bins = np.arange(-20, 20, 0.5))
plt.xlabel("ReadPosRankSum")
plt.ylabel("Frequency")
plt.xlim(-12, 12)
plt.axvline(-10, color = 'r')
plt.axvline(10, color = 'r')

plt.tight_layout()
plt.show()
display()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3160555503398416> in <module> () 
 ----> 1 variantqc_table = vds_gDP_gGQ_vCR_sDP_sGT . variants_keytable ( ) . to_pandas ( ) 
 2 
 3 plt . clf ( ) # clears the entire current figure with all its axes, but leaves the window opened, such that it may be reused for other plots. 
 4 plt . figure ( figsize = ( 7 , 7 ) ) # figure size in inches; change according to your screen size and resolution 
 5 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/keytable.py in to_pandas (self, expand, flatten) 
 533 : return : pandas . DataFrame 
 534 """
 --> 535 return self . to_dataframe ( expand , flatten ) . toPandas ( ) 
 536 
 537 def export_mongodb ( self , mode = 'append' ) : 

 /databricks/spark/python/pyspark/sql/dataframe.py in toPandas (self) 
 2127 import pandas as pd 
 2128 
 -> 2129 if self . sql_ctx . _conf . pandasRespectSessionTimeZone ( ) : 
 2130 timezone = self . sql_ctx . _conf . sessionLocalTimeZone ( ) 
 2131 else : 

 /databricks/spark/python/pyspark/sql/context.py in _conf (self) 
 97 def _conf ( self ) : 
 98 """Accessor for the JVM SQL-specific configurations""" 
 ---> 99 return self . sparkSession . _jsparkSession . sessionState ( ) . conf ( ) 
 100 
 101 @ classmethod 

 AttributeError : 'JavaMember' object has no attribute 'sessionState'

Lastly we use the [filter_variants_expr](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.filter_variants_expr) method to keep only those variants that meet the cut-off requirements (red lines in the plots above).

In [81]:
vds_gDP_gGQ_vCR_sDP_sGT_vFilter = vds_gDP_gGQ_vCR_sDP_sGT.filter_variants_expr('va.info.MQ >= 30.00 && va.info.SOR <= 4.000 && va.info.QD >= 2.00 && va.info.FS <= 60.000 && va.info.MQRankSum >= -20.000 && va.info.ReadPosRankSum >= -10.000 && va.info.ReadPosRankSum <= 10.000')
print('variants before filtering: %d' % vds_gDP_gGQ_vCR_sDP_sGT.count_variants())
print('variants after filtering: %d' % vds_gDP_gGQ_vCR_sDP_sGT_vFilter.count_variants())

variants before filtering: 49993
variants after filtering: 44368

Verify the filtering results with plots:

In [83]:
# https://discuss.hail.is/t/potential-compatibility-issue-with-spark-2-2-0/417/3

In [84]:
variantqc_table = vds_gDP_gGQ_vCR_sDP_sGT_vFilter.variants_keytable().to_pandas()

plt.clf() # clears the entire current figure with all its axes, but leaves the window opened, such that it may be reused for other plots.
plt.figure(figsize=(7,7)) # figure size in inches; change according to your screen size and resolution

plt.subplot(3, 2, 1)
variantgq_MQ = variantqc_table["va.info.MQ"]
plt.hist(variantgq_MQ.dropna(), bins = np.arange(0, 101, 2))
plt.xlabel("MQ")
plt.ylabel("Frequency")
plt.xlim(0, 100)
plt.axvline(30, color = 'r')

plt.subplot(3, 2, 2)
variantgq_SOR = variantqc_table["va.info.SOR"]
plt.hist(variantgq_SOR.dropna(), bins = np.arange(0, 8, 0.2))
plt.xlabel("SOR")
plt.ylabel("Frequency")
plt.xlim(0, 8)
plt.axvline(4, color = 'r')

plt.subplot(3, 2, 3)
variantgq_QD = variantqc_table["va.info.QD"]
plt.hist(variantgq_QD.dropna(), bins = np.arange(0, 40, 1))
plt.xlabel("QD")
plt.ylabel("Frequency")
plt.xlim(0, 40)
plt.axvline(2, color = 'r')

plt.subplot(3, 2, 4)
variantgq_FS = variantqc_table["va.info.FS"]
plt.hist(variantgq_FS.dropna(), bins = np.arange(0, 100, 2))
plt.xlabel("FS")
plt.ylabel("Frequency")
plt.xlim(0, 100)
plt.axvline(60, color = 'r')

plt.subplot(3, 2, 5)
variantgq_MQRankSum = variantqc_table["va.info.MQRankSum"]
plt.hist(variantgq_MQRankSum.dropna(), bins = np.arange(-20, 20, 1))
plt.xlabel("MQRankSum")
plt.ylabel("Frequency")
plt.xlim(-25, 25)
plt.axvline(-20, color = 'r')

plt.subplot(3, 2, 6)
variantgq_ReadPosRankSum = variantqc_table["va.info.ReadPosRankSum"]
plt.hist(variantgq_ReadPosRankSum.dropna(), bins = np.arange(-20, 20, 0.5))
plt.xlabel("ReadPosRankSum")
plt.ylabel("Frequency")
plt.xlim(-12, 12)
plt.axvline(-10, color = 'r')
plt.axvline(10, color = 'r')

plt.tight_layout()
plt.show()
display()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3160555503398420> in <module> () 
 ----> 1 variantqc_table = vds_gDP_gGQ_vCR_sDP_sGT_vFilter . variants_keytable ( ) . to_pandas ( ) 
 2 
 3 plt . clf ( ) # clears the entire current figure with all its axes, but leaves the window opened, such that it may be reused for other plots. 
 4 plt . figure ( figsize = ( 7 , 7 ) ) # figure size in inches; change according to your screen size and resolution 
 5 

 /local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/keytable.py in to_pandas (self, expand, flatten) 
 533 : return : pandas . DataFrame 
 534 """
 --> 535 return self . to_dataframe ( expand , flatten ) . toPandas ( ) 
 536 
 537 def export_mongodb ( self , mode = 'append' ) : 

 /databricks/spark/python/pyspark/sql/dataframe.py in toPandas (self) 
 2127 import pandas as pd 
 2128 
 -> 2129 if self . sql_ctx . _conf . pandasRespectSessionTimeZone ( ) : 
 2130 timezone = self . sql_ctx . _conf . sessionLocalTimeZone ( ) 
 2131 else : 

 /databricks/spark/python/pyspark/sql/context.py in _conf (self) 
 97 def _conf ( self ) : 
 98 """Accessor for the JVM SQL-specific configurations""" 
 ---> 99 return self . sparkSession . _jsparkSession . sessionState ( ) . conf ( ) 
 100 
 101 @ classmethod 

 AttributeError : 'JavaMember' object has no attribute 'sessionState'

## PCA

To check if there is any genetic structure, we will use a principal component analysis (PCA).

In [86]:
vds_pca = (vds_gDP_gGQ_vCR_sDP_sGT_vFilter.pca(scores='sa.pca'))

We can then make a Python plot of the samples in PC space colored by population group:

In [88]:
pca_table = vds_pca.samples_keytable().to_pandas()
colors = {'ASI': 'green', 'EUR': 'red', 'ME': 'blue'}
plt.clf()
plt.figure(figsize=(7,7)) # figure size in inches; change according to your screen size and resolution
plt.scatter(pca_table["sa.pca.PC1"], pca_table["sa.pca.PC2"], c = pca_table["sa.myAnnot.Population"].map(colors), alpha = .7, s=100)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.xlim(-0.7, 0.7)
plt.ylim(-0.7, 0.7)
legend_entries = [mpatches.Patch(color= c, label=myAnnot) for myAnnot, c in colors.items()]
plt.legend(handles=legend_entries, prop={'size': 15})
plt.show()
display()

This plot reflects the known [population structure in *Capsella bursa-pastoris*](http://onlinelibrary.wiley.com/doi/10.1111/mec.13491/full).

You can see the location of these samples with some R code:

Install packages that are necessary to produce a geographic map:

In [91]:
%r 
install.packages(c("maps", "rworldmap"))

In [92]:
%r
library(rworldmap)
library(maps)

geo <- read.table("/databricks/driver/Cbp31_annot.csv", header = T) # read file not from dbfs

newmap <- getMap(resolution = "hight") # create a map object

pchS <- c(rep(15, 11), rep(19, 13), rep(17, 7)) # designate genetic clusters with different colours
colS <- c(rep("green", 11), rep("red", 13), rep("blue", 7))
groupsL <- c('ASI', 'EUR', 'ME')
pchL <- c(15, 19, 17)
colL <- c("green", "red", "blue")

par(mar=c(3, 3, 2, 2), cex=1)
plot(newmap, xlim = c(-40, 140), ylim = c(20, 50), asp = 1, bg='#DCDCDC', lty=3, lwd=0.3, col="#ffffff")
map.axes()
points(x=geo$Longitude, y=geo$Latitude, pch=pchS, cex=1.2, col=colS)
legend("topright", leg=groupsL, pch=pchL, col=colL, ncol=1, pt.cex=1.2, bg="#ffffff")

## Summary

Data filtering:
 - Filter genotypes
 - Filter samples
 - Filter variants

*Variants can be filtered before samples filtering if samples are of greater priority in a study.*

Such genetic data can be analyzed in various ways. A PCA is just one simple example.

## Additional

It is recommended to go through the original [Analysis of 1000 Genomes with Spark and Hail](https://docs.databricks.com/spark/latest/training/1000-genomes.html).

You can also read [Hail Overview](https://www.hail.is/hail/overview.html), look through the [Hail objects](https://www.hail.is/hail/hail_objects.html) representing many core concepts in genetics, and check out the many Hail functions defined in the [Python API](https://hail.is/hail/api.html).

# Shifting Gear - Another Sample

In [97]:
#

In [98]:
from pprint import pprint
hl.plot.output_notebook()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3160555503398448> in <module> () 
 1 from pprint import pprint 
 ----> 2 hl . plot . output_notebook ( ) 

 NameError : name 'hl' is not defined

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3160555503398449> in <module> () 
 ----> 1 hl 

 NameError : name 'hl' is not defined

In [100]:
hl

Out[ 108 ]: <module 'hail' from '/local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/__init__.py'>

In [101]:
hc

Out[ 109 ]: <hail.context.HailContext at 0x7f468c11aba8>

In [102]:

from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()


--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-3160555503398452> in <module> () 
 1 
 ----> 2 from hail . plot import show 
 3 from pprint import pprint 
 4 hl . plot . output_notebook ( ) 

 ImportError : No module named 'hail.plot'

In [103]:
hl

Out[ 111 ]: <module 'hail' from '/local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/__init__.py'>

In [104]:
print(type(hl))

<class 'module'>

In [105]:
hl.utils.get_1kg('data/')

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3160555503398455> in <module> () 
 ----> 1 hl . utils . get_1kg ( 'data/' ) 

 AttributeError : module 'hail.utils' has no attribute 'get_1kg'

* https://hail.is/docs/0.2/tutorials/01-genome-wide-association-study.html

In [107]:
dir(hl.stats)

Out[ 114 ]: 
['BetaDist',
 'TruncatedBetaDist',
 'UniformDist',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'env']

In [108]:
hl.stats

Out[ 115 ]: <module 'hail.stats' from '/local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/stats.py'>

In [109]:
print(type(hl.stats))

<class 'module'>

In [110]:

import hail as hl

dir(hl)



Out[ 121 ]: 
['HailContext',
 'KeyTable',
 'TextTableConfig',
 'Type',
 'VariantDataset',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'context',
 'dataset',
 'java',
 'keytable',
 'representation',
 'stats',
 'type',
 'utils']

In [111]:
hl.context

Out[ 122 ]: <module 'hail.context' from '/local_disk0/spark-c36517b3-41eb-4014-9f4a-da104c7728fd/userFiles-dbab7a83-dab7-42cf-9298-3d330d11309e/addedFile2248754259193548425c7eaedcd_8828_4b98_8cc6_90caba462aa4_hail_devel_py2_7_databricks_20ed0-8a939.egg/hail/context.py'>

In [112]:
print(type(hl.context))

<class 'module'>

In [113]:
dir(hl.utils)

Out[ 124 ]: 
['Py4JJavaError',
 'TextTableConfig',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'env',
 'raise_py4j_exception']

In [114]:

import hail as hl
hl.init(sc)



--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3160555503398464> in <module> () 
 1 
 2 import hail as hl 
 ----> 3 hl . init ( sc ) 
 4 

 AttributeError : module 'hail' has no attribute 'init'